#git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/Deep Learning/final_project/20242R0136COSE47402

In [ ]:
!git config --global user.email 'robosun78@korea.ac.kr'
!git config --global user.name 'robosun78'

In [ ]:
!ls

In [ ]:
!git add FinalProject

In [ ]:
!git commit -m 'CLIP implement'
!git push

[main fa8d7f5] CLIP implement
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite FinalProject/Final_proejct_2022320027.ipynb (93%)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 141.41 KiB | 4.88 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/robosun78/20242R0136COSE47402.git
   5c4dcec..fa8d7f5  main -> main


#Code

In [4]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 88.4 MB/s eta 0:00:00


In [8]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 85.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3


In [1]:
import torch
import torchvision
from transformers import AutoModel, AutoImageProcessor
from torchvision import transforms
import matplotlib.pyplot as plt
import os
import numpy as np
from PIL import Image
import faiss

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

##CLIP

In [3]:
from transformers import CLIPProcessor, CLIPModel

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.10k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/599M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [7]:
def extract_features_clip(image):
  with torch.no_grad():
      inputs_clip = clip_processor(images=image, return_tensors="pt").to(device)
      image_features_clip = clip_model.get_image_features(**inputs_clip)
      return image_features_clip

## DinoV2

In [5]:
from transformers import AutoImageProcessor, AutoModel

#dinov2_ = torch.hub.load("facebookresearch/dinov2", "dinov2_vitb14")
dino_model = AutoModel.from_pretrained('facebook/dinov2-base')
dino_processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')

config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

In [8]:
def extract_features_dino(image):
  with torch.no_grad():
      inputs_dino = dino_processor(images=image, return_tensors="pt").to(device)
      outputs_dino = dino_model(**inputs_dino)
      image_features_dino = outputs_dino.last_hidden_state
      image_features_dino = image_features_dino.mean(dim=1)
      return image_features_dino


## ViT

# Simiarity

## image embeddings

In [ ]:
images = []
for root, dirs, files in os.walk('./val2017/'):
    for file in files:
        if file.endswith('jpg'):
            images.append(root  + '/'+ file)

In [ ]:
def normalizeL2(embeddings):
    vector = embeddings.detach().cpu().numpy()
    vector = np.float32(vector)
    faiss.normalize_L2(vector)
    return vector

In [ ]:
def make_embeddings(embedding, index):
    vector = embedding.detach().cpu().numpy()
    vector = np.float32(vector)
    faiss.normalize_L2(vector)
    index.add(vector)

In [ ]:
index_clip = faiss.IndexFlatL2(512)
index_dino = faiss.IndexFlatL2(768)

In [ ]:
for image_path in images:
    img = Image.open(image_path).convert('RGB')
    clip_features = extract_features_clip(img)
    make_embeddings(clip_features,index_clip)
    dino_features = extract_features_dino(img)
    make_embeddings(dino_features,index_dino)

In [ ]:
faiss.write_index(index_clip,"clip.index")
faiss.write_index(index_dino,"dino.index")

## test

In [ ]:
img = ''

In [ ]:
dino_features = extract_features_dino(img)
image_features_dino = normalizeL2(dino_features)
clip_features = extract_features_clip(img)
image_features_clip = normalizeL2(clip_features)

In [ ]:
index_clip = faiss.read_index("clip.index")
index_dino = faiss.read_index("dino.index")

In [ ]:
d_dino,i_dino = index_dino.search(image_features_dino,5)
d_clip,i_clip = index_clip.search(image_features_clip,5)